# Tic-Tac-Toe Game 

Let me walk you through how I built a tic tac toe from scratch in python

Importing the necessary classes beforehand:

In [ ]:
import numpy as np
import random
import pandas as pd
from IPython.display import clear_output, HTML
from time import sleep
pd.set_option('mode.chained_assignment', None)

## GameBoard Class

The GameBoard Class represents a 3 x 3 game board for a tic tac toe game. The GameBoard Class has the following member variables and functions:

#### Member variables:-
- _board_: a 3x3 numpy array representing the actual game board in the back-end.
- _key_: a key which helps translate the back-end game board to the visual classic tic tac toe board.

#### Member functions:-
- _constructor (\_\_init\__)_: initializes the member variables.
- _call (\_\_call_\_\)_: prints the translated game board when the object is called.
- _check_spot_: returns 0 if the spot in the board has previously been filled in the game, else returns 1.
- _fill_spot_: fills an empty spot in the board with the desired value. It also prints the board after the play is executed.
- _check-win_: returns 1 if the first player has won the game, returns -1 if the second player has won the game, else returns 0 to continue the game.
- _check-tie_: returns _True_ if the game has ended as a tie, else returns _False_.

In [ ]:
class GameBoard:
    def __init__(self):
        self.board = np.zeros((3,3))
        self.key = {1 : 'X', -1 : 'O', 0 : '_'}
        
    def __call__(self):      
        for row in self.board:
            print(self.key[row[0]] + ' ' + self.key[row[1]]+ ' ' + self.key[row[2]])
        print('\n')
            
    def check_spot(self, play):
        if self.board[play] != 0:   # Spot is filled => Invalid play
            return 0
        return 1                    # Spot is available => Valid play
    
    def fill_spot(self, play, value = -1):
        self.board[play] = value
        self()
    
    def check_win(self):
        # Column and Row win
        if (3 in self.board.sum(axis=0)) or (3 in self.board.sum(axis=1)):          
            return 1                                                               
        elif (-3 in self.board.sum(axis=0)) or (-3 in self.board.sum(axis=1)):
            return -1
    
        # Cross win
        s1 = self.board[0,0] + self.board[1,1] + self.board[2,2];
        s2 = self.board[0,2] + self.board[1,1] + self.board[2,0];
        
        if ((s1 == 3) or (s2 == 3)):
            return 1
        elif ((s1 == -3) or (s2 == -3)):
            return -1  
        
        # 1  : Player1 has won => End Game
        # -1 : Player2 has won => End Game
        # 0  : No one has won yet => Continue Game    
        return 0 
    
    def check_tie(self):
        if not(0 in self.board):
            return True           # Game has ended as a tie
        return False              # Game hasn't ended yet

## Player Class

The Player Class represents a player in the classical tic tac toe game. It has the following member variables and functions:

#### Member variables:-
- _translate_: a dictionary used to translate the player's input to a specific location on the board.
- _character_: an integer representing the player's move in the back-end.
- _computer_: a boolean denoting whether the player is a computer or not.
- _name_: the player's chosen Username.

#### Member functions:-
- _user_record (\_\_init\__)_: checks if the player's username is already present in the scoresheet. If not, it adds the username to it.
- _call (\_\_call_\_\)_: prompts the player to play a move by selecting a spot on the game board and returns th play as location index. 
- _user_win_: prints the player as the winner and updates the scoresheet by adding a win to the username's "Wins" count.
- _user_loss_: updates the scoresheet by adding a loss to the username's "Losses" count.

In [ ]:
class Player:
    def __init__(self, computer=False, character=-1):
        self.translate = {'Q':(0,0) , 'W':(0,1) , 'E':(0,2),
                          'A':(1,0) , 'S':(1,1) , 'D':(1,2),
                          'Z':(2,0) , 'X':(2,1) , 'C':(2,2)}
        self.character = character                         # 1  => X  
        self.computer = computer                           # -1 => O
        
        clear_output()
        if computer == False:
            self.name = input("Enter your Username(case-sensitive):  ")
            self.user_record()
            sleep(3)
            
    def user_record(self):
        df = pd.read_csv('Scoresheet.csv')
        if sum(df["Username"] == self.name):                              # Checks if a player with the same username
            print(f"Welcome Back {self.name}!\n\n Your previous record:\n") # has played before. If yes, it displays the  
            print(df[["Wins","Losses"]][df["Username"]==self.name])       # players wins and losses 
        else:
            print(f"Welcome {self.name}!")                                # If it is a new username, adds a record to
            df = df.append({"Username": self.name,                        # the scoresheet
                       "Wins": 0,
                       "Losses": 0}, ignore_index=True)
        
        df.to_csv('Scoresheet.csv', index=None)
    
    def __call__(self):
        if self.computer == False:
            valid_inputs = list(self.translate.keys())

            print(f"{self.name}'s Turn:")
            play = input("Enter your play (according to the format):   ")
            while not(play in valid_inputs): 
                play = input("Invalid input. Keep in mind the format shown in the beginning and re-enter:  ")
            play = self.translate[play]
        
        else:
            play = (random.randint(0,2), random.randint(0,2))            # The computer randomly makes a play
       
        return play
    
    def user_win(self):
        df= pd.read_csv("Scoresheet.csv")
        if self.computer == False:
            df["Wins"][df["Username"]==self.name] += 1
            print(f"\nCONGRATS {self.name}! YOU HAVE WON THE GAME!!")
        
        else:
            print("\nOh no, looks like the COMPUTER won!")
    
        df.to_csv("Scoresheet.csv", index=None)
        
        
    def user_loss(self):
        df= pd.read_csv("Scoresheet.csv")
        if self.computer == False:
            df["Losses"][df["Username"]==self.name] += 1
    
        df.to_csv("Scoresheet.csv", index=None)
        

## Playing the game!

To initiate the game, I have declared 2 functions:
- _intro_: clears the output screen and prints the introduction with the rules. It also offers a choice to the user to either play against the computer or with another player. Returns a value corresponding to the choice of the user.
- _lets_play_: initializes objects and plays the game till it ends as a tie or win.

In [ ]:
def intro():
    clear_output()
    print("TTT  III  CCC    TTT   A   CCC    TTT  OOO  EEE")
    print(" T    I   C       T   A A  C       T   O O  E")
    print(" T   III  CCC     T  A   A CCC     T   OOO  EEE")
    
    print(f"\nWelcome to the game of Tic-Tac-Toe!\n\nRules of the game:")
    print("1. Classic tic tac toe rules")
    print("2. There are a total of 9 spaces to fill in a tic tac toe board")
    print("3. Use the corresponding keys on your keyboard (remember to keep CAPS LOCK ON) to fill the board:")
    print("   Q W E   ->   _ _ _\n   A S D   ->   _ _ _\n   Z X C   ->   _ _ _")
    print(f"\nTake a few more seconds to grasp the format and then we'll move on.")           
    
    sleep(2)
    
    computer = input("\nJust one last thing: \nEnter Y if you want play against the computer, else \nEnter N if you want to play a 2 player game: \n")
    while not(computer in ['Y', 'N']):
        computer = input("Invalid input! Enter Y/N (case sensitive): ")
    
    sleep(2)
    return computer

In [ ]:
def lets_play():
    computer = intro()
    
    
    Board = GameBoard()
    Player1 = Player(False,1)
    Player2 = Player(computer=='Y',-1)
    
    print("Let's Begin Playing the Game!!")
    sleep(5)
    
    while (not(Board.check_win())):
        clear_output()
        print("Current Game Board:")
        Board()
        
        play = Player1()
        while Board.check_spot(play)==0:
            print("That spot has already been filled! Try some other spot!  ")
            play = Player1()
        Board.fill_spot(play, Player1.character)
        
        if Board.check_win() == 1:
            Player1.user_win()
            Player2.user_loss()
            break
        
        if Board.check_tie():
            print("Looks like the game ended as a tie and no one won :(")
            break
            
        if Player2.computer:
            print("Computer's Turn:")
            sleep(1)
        play = Player2()
        while Board.check_spot(play)==0:
            if Player2.computer == False:
                print("That spot has already been filled! Try some other spot!  ")
            play = Player2()    
        Board.fill_spot(play, Player2.character)
        
        if Board.check_win() == -1:
            Player1.user_loss()
            Player2.user_win()
            break
            
        sleep(3)
    
    sleep(3)
    clear_output()
    print("Current Scoresheet:\n")
    df= pd.read_csv("Scoresheet.csv")
    print(df)

### Run the cell below to play the game!

In [ ]:
lets_play()

## Possible improvments to the game:

Given the time constraint, this is the best game I can come up with. However, personally, I believe this game could have been improved with more knowledge and time at hand. The ideas that come to my mind as of now are: 
1. **Implementing Reinforcement Learning(RL)**- RL models can be used to train the computer player to make it a more real competitor. At this stage the computer plays randomly and is fairly easy to defeat.
2. **Game Levels**- Having trained the computer with RL, we could add features relating to the level of the bot, say, Easy, Medium and Hard.
3. **Improving the User Experience**- There is always room for improvement in the user interface in any application. 